In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
torch.manual_seed(0)

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
#configuring the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# ...

# mnist train_test datasets
train_ds = torchvision.datasets.MNIST(root="./data", train=True, transform=transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
]), download=True)
test_ds = torchvision.datasets.MNIST(root="./data", train=False, transform=transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
]), download=True)

# ...


100%|██████████| 9912422/9912422 [00:00<00:00, 80444621.85it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 89597406.67it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 22334687.76it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 18126097.78it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
#data_loader
train_d1 = torch.utils.data.DataLoader(dataset = train_ds, batch_size=100, shuffle = True)
test_d1 = torch.utils.data.DataLoader(dataset = train_ds, batch_size=100, shuffle = False)


In [ ]:



class VGG16(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG16, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


model = VGG16(num_classes = 10).to(device)
print(model)




VGG16(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation

In [ ]:
# loss and optimizer

losses = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr = 0.01)

#training

num_epochs = 30

epochs = tqdm(range(num_epochs))

for epoch in epochs:
  train_loss = []
  model.train()

  for batch in (train_d1):
    optim.zero_grad()
    images = batch[0].reshape(-1, 28*28).to(device)
    label = batch[1].to(device)

    #forward
    output = model(images)
    loss = torch.nn.CrossEntropyLoss()(output.squeeze(-1), label)

    #backward pass and optimize
    loss.backward()
    optim.step()
    train_loss.append(loss.item())

  loss_now = np.mean(train_loss)
  epochs.set_postfix({"loss": loss_now})



  0%|          | 0/30 [00:00<?, ?it/s]

RuntimeError: ignored

In [ ]:
#Testing section
predlist = torch.zeros(0, dtype = torch.long, device = "cpu")
lbllist = torch.zeros(0, dtype = torch.long, device = "cpu")

with torch.no_grad():
  for ims, labels in test_d1:
    ims = ims.reshape(-1, 28*28).to(device)
    labels = labels.to(device)
    outputs = model(ims)
    _, predicted = torch.max(outputs.data, 1)

    # append batch predicttion results
    predlist = torch.cat([predlist, predicted.view(-1).cpu()])
    lbllist = torch.cat([lbllist, labels.view(-1).cpu()])



In [ ]:
#confusion matrix

conf_mat = confusion_matrix(np.asarray(lbllist), np.asarray(predlist))
print(conf_mat)

#per class accuracy
class_accuracy = 100*conf_mat.diagonal()/conf_mat.sum(1)
print(class_accuracy)

#report
print(classification_report(np.asarray(lbllist), np.asarray(predlist), digits = 3))

In [ ]:
sns.heatmap(conf_mat)
plt.show()